In [2]:
train = pd.read_csv('./train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./test.csv', encoding = 'utf-8-sig')

samples = []

for i in range(10):
    sample = f"input : {train['input'][i]} \n output : {train['output'][i]}"
    samples.append(sample)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_id = 'beomi/gemma-ko-7b'
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [ ]:
pipe = pipeline(
    task="text-generation",
    model=model, 
    tokenizer=tokenizer 
)

restored_reviews = []


for index, row in test.iterrows():
    query = row['input']  
    
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant specializing in restoring obfuscated Korean reviews. "
                "Your task is to transform the given obfuscated Korean review into a clear, correct, "
                "and natural-sounding Korean review that reflects its original meaning. "
                "Below are examples of obfuscated Korean reviews and their restored forms:\n\n"
                f"Example, {samples}"  
                "Spacing and word length in the output must be restored to the same as in the input. "
                "Do not provide any description. Print only in Korean."
            )
        },
        {
            "role": "user",
            "content": f"input : {query}, output : "
        },
    ]
    
    prompt = "\n".join([m["content"] for m in messages]).strip()
    

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
        max_new_tokens=len(query),
        eos_token_id=pipe.tokenizer.eos_token_id
    )
    
    generated_text = outputs[0]['generated_text']
    result = generated_text[len(prompt):].strip()
        

    restored_reviews.append(result)

In [21]:
rc=pd.read_csv('test.csv')

max=0

for i, row in rc.iterrows():
    if len(row['input'])>max:
        max=len(row['input'])
print(max)

1965


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import time

# 샘플 데이터셋 생성 (50개 샘플)
sample_data = {
    "input": [
        "This is a test input for checking the tokenization process.",
        "Another sample input, short and simple.",
        "Here is a longer sentence that will require more tokens for the model to process.",
        "The quick brown fox jumps over the lazy dog.",
        "Sample input with punctuation! How will the tokenizer handle it?",
        "This is a test sentence to verify batch processing behavior.",
        "Longer input designed to check if padding or truncation works as expected when used in tokenization.",
        "Data augmentation helps improve model performance.",
        "Test sentence to ensure proper padding and truncation works.",
        "Short input to check the efficiency of tokenization.",
        "This example is used for testing whether truncation occurs when tokens exceed the max length.",
        "Random sentence to observe how model processes the input efficiently.",
        "Text tokenization is an essential process for NLP models.",
        "Padding tokens are added to match the sequence length.",
        "This is a simple test to check tokenizer behavior with long texts.",
        "Tokenizers need to handle punctuation marks carefully.",
        "Using padding and truncation strategies effectively ensures model stability.",
        "Input with several sentences to test batch processing capacity.",
        "Short sentences are often used in fast model evaluation.",
        "Longer inputs with multi-token words help check model handling capacity.",
        "Sentences of varied length improve tokenizers' ability to generalize.",
        "Tokenizer efficiency is crucial for large-scale training.",
        "Model needs sufficient data to generalize across varied inputs.",
        "Using multiple samples in training aids model robustness.",
        "Test input with various punctuation to see if tokenizer handles it correctly.",
        "This input sentence is designed to be very long and check for truncation.",
        "This sentence contains multiple clauses to check for tokenization errors.",
        "Longer sentences should trigger proper truncation behavior in models.",
        "Short inputs require minimal padding to fit within token limits.",
        "Medium-length sentences are ideal for checking padding behavior.",
        "Using long sentences in NLP models ensures the tokenizer works efficiently.",
        "Balanced sentences help improve model performance during training.",
        "Test sentences of different lengths help improve batch processing capabilities.",
        "Padding and truncation methods help ensure model accuracy on various inputs.",
        "Truncating long sentences ensures the model doesn't exceed memory limits.",
        "Shorter inputs ensure the model is capable of handling large batches.",
        "The sentence length affects both memory consumption and model accuracy.",
        "Using a mix of sentence lengths improves model training efficiency.",
        "Appropriate padding and truncation methods help enhance the model's effectiveness.",
        "Correct tokenization is vital for any NLP model's functionality.",
        "Batch processing enables efficient handling of large datasets.",
        "Optimized tokenization leads to faster training times in models.",
        "Advanced models require efficient tokenizers to process large-scale data.",
        "Tokenization must be precise to avoid loss of important information.",
        "Sentence splitting is a crucial process for preparing input data for models.",
        "For long inputs, truncation is essential to prevent memory overflow.",
        "Efficient padding helps reduce unnecessary computation during model training.",
        "Effective truncation ensures the model only processes relevant parts of the input."
    ],
    "output": [
        "This is a test output for the first input.",
        "Output for the second input here.",
        "The model should process the longer sentence with more tokens.",
        "The fox jumped over the dog, according to the story.",
        "Processed output from a sentence with punctuation.",
        "A test output that matches the input sentence length.",
        "Testing longer input with padding and truncation in mind.",
        "Data augmentation improves performance by diversifying inputs.",
        "Output for a sentence that tests padding and truncation strategies.",
        "A quick test for short input processing.",
        "A test for truncation when input tokens exceed max length.",
        "Random output generated to observe model behavior.",
        "Output related to tokenization process in NLP models.",
        "Padding tokens ensure sequences are aligned for model processing.",
        "Output matches the input sentence's token length after processing.",
        "Efficient tokenization ensures faster training and fewer errors.",
        "Longer sentences are processed correctly using proper tokenization.",
        "Padding and truncation strategies are confirmed to work efficiently.",
        "Batch processing in NLP tasks enhances speed and stability.",
        "Output verification for smaller input sentences.",
        "Longer inputs generate more tokens and require careful tokenization.",
        "Generalization tests are easier with varied sentence lengths.",
        "Improved tokenizer efficiency ensures stability in large-scale tasks.",
        "Model stability is confirmed through multiple input variations.",
        "Tokenizer handles input variations efficiently for robust model behavior.",
        "Appropriate sentence structures improve model generalization.",
        "The truncation of long sentences ensures accuracy without exceeding memory limits.",
        "Multiple clauses test the tokenizer's handling capacity in NLP tasks.",
        "Tokenization works as expected for both long and short sentences.",
        "Short sentences optimize model performance during batch processing.",
        "Efficient tokenization ensures minimal resource usage during training.",
        "Balancing input lengths optimizes the model's capacity to generalize.",
        "Generalization during training is enhanced by varying sentence lengths.",
        "Padding and truncation support effective model accuracy for various tasks.",
        "Proper tokenization ensures minimal loss of information during training.",
        "Batch processing helps models handle large datasets quickly and accurately.",
        "Optimized tokenization techniques support faster and more accurate model training.",
        "Truncation is essential for processing large inputs without overloading memory.",
        "Truncating long sentences maintains memory efficiency while ensuring accuracy.",
        "Shorter inputs ensure efficient computation during large batch training.",
        "Model accuracy is not compromised by varying input lengths during training.",
        "Training efficiency is improved by managing sentence lengths appropriately.",
        "Optimized truncation ensures that only relevant information is processed.",
        "Efficient tokenization and truncation methods enhance the model's effectiveness."
    ]
}

# DataFrame 생성 후 Datasets 형식으로 변환
sample_df = pd.DataFrame(sample_data)
dataset = Dataset.from_pandas(sample_df)

# 모델과 토크나이저 로드
MODEL_NAME = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Tokenizer 적용
def tokenize_function(examples):
    return tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)

# 데이터셋 토큰화
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 배치 내 개별 길이 대응 함수
def get_max_new_tokens(batch):
    max_input_length = max([len(tokenizer(x)["input_ids"]) for x in batch["input"]])
    print(f"Max input length in this batch: {max_input_length}")  # 출력 추가
    return max_input_length

# Trainer 콜백을 사용하여 배치 내 길이 기반 `max_new_tokens` 동적 조정
class DynamicMaxTokensCallback(TrainerCallback):
    def on_batch_begin(self, args, state, control, **kwargs):
        max_new_tokens = get_max_new_tokens(kwargs['inputs'])  # 배치 내 가장 긴 길이 추출
        control.max_new_tokens = max_new_tokens
        print(f"Dynamic max_new_tokens: {max_new_tokens}")  # 출력 추가
        return control

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,  # 기본 배치 사이즈
    gradient_accumulation_steps=8,  # Effective batch size
    fp16=True,  # FP16 사용
    save_steps=1000,
    logging_steps=50,
    evaluation_strategy="steps",
    num_train_epochs=3,
    save_total_limit=2,
    gradient_checkpointing=True,  # Gradient Checkpointing 활성화
    learning_rate=2e-5
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
    callbacks=[DynamicMaxTokensCallback()]  # 콜백으로 동적 `max_new_tokens` 적용
)

# 실험 시작 전 시간 측정
start_time = time.time()

# 학습 시작 (샘플 데이터로 학습)
trainer.train()

# 학습 시간 측정
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Total training time for sample dataset: {elapsed_time / 60:.2f} minutes")


In [ ]:
import pandas as pd
from konoise import NoiseGenerator

# train.csv 파일 불러오기
train_df = pd.read_csv("train.csv")

# NoiseGenerator 객체 초기화 (konoise)
generator = NoiseGenerator()

# 증강된 데이터 저장할 리스트
augmented_data = []

# 각 데이터에 대해 증강
for idx, row in train_df.iterrows():
    input_text = row['input']
    output_text = row['output']
    
    # input 텍스트에 대해 노이즈 추가 (난독화된 텍스트 생성)
    augmented_input_list = generator.generate(input_text, methods='disattach-letters', prob=1.0)
    
    # 리스트에서 첫 번째 항목을 가져와서 슬라이싱
    augmented_input = augmented_input_list[0][0]
    
    # output 텍스트는 그대로 교정된 상태로 유지
    augmented_output = output_text.strip()  # 교정된 텍스트
    
    # 증강된 데이터를 리스트에 저장
    augmented_data.append([augmented_input, augmented_output])

# 증강된 데이터를 새로운 DataFrame에 저장
augmented_df = pd.DataFrame(augmented_data, columns=['input', 'output'])

# 증강된 데이터만 저장한 CSV 파일
augmented_df.to_csv("augmented_data.csv", index=False)

# 기존 데이터와 증강된 데이터를 합친 CSV 파일
combined_df = pd.concat([train_df[['input', 'output']], augmented_df], ignore_index=True)
combined_df.to_csv("combined_data.csv", index=False)
